In [ ]:
from localpdb import PDB
import gzip

lpdb = PDB(db_path='./lpdb_data')
lpdb.entries = lpdb.entries.query('type == "prot"') # Protein structures
lpdb.entries = lpdb.entries.query('method == "diffraction"') # solved with X-ray diffraction
lpdb.entries = lpdb.entries.query('resolution <= 2.5') # with resolution below 2.5A
lpdb.entries = lpdb.entries.query('deposition_date.dt.year >= 2010') # added after 2010
lpdb.chains = lpdb.chains.query('ncbi_taxid == "9606"') # human proteins
# lpdb.ecod = lpdb.ecod.query('t_name == "S-adenosyl-L-methionine-dependent methyltransferases"') # SAM dependent methyltransferases

# Remove redundancy (select only representative structure from each sequence cluster)
lpdb.load_clustering_data(redundancy=90)
lpdb.chains = lpdb.chains[lpdb.chains['clust-90'].notnull()]

representative = lpdb.chains.groupby(by='clust-90')['resolution'].idxmin()
lpdb.chains = lpdb.chains.loc[representative]

lpdb.chains.to_csv('dataset.csv') # Save dataset